In [48]:
import random
from functools import lru_cache
import os
import math

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [33]:
movies_data = pd.read_csv('Data/movies.csv')
ratings_data = pd.read_csv('Data/ratings.csv')
tags_data = pd.read_csv('Data/tags.csv')
links = pd.read_csv('Data/links.csv')
genome_tags_data = pd.read_csv('Data/genome-tags.csv')
genome_scores_data = pd.read_csv('Data/genome-scores.csv')


In [3]:
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies_data.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [5]:
ratings_data.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [11]:
ratings_data.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

### creating a new table having movie_title, year, avg_rating as columns

In [8]:
#df = pd.read_csv('ratings.csv', usecols=['userId', 'movieId', 'rating'])

In [9]:
#timestamp column is not required
#df

In [10]:
def munge_title(title):
    i = title.rfind(' (')
    if i != -1:
        title = title[:i]
    for suff_word in ['The', 'A', 'An']:
        suffix = ', {}'.format(suff_word)
        if title.endswith(suffix):
            title = suff_word + ' ' + title[:-len(suffix)]
    return title

In [13]:
#function to get the year of all the movies
def get_year(title):
    l = title.rfind('(') + 1  ## rfind finds the last occurence 
    try:
        return int(title[l:l+4])
    except ValueError:
        #print(title, end='\t')
        return 0

In [14]:
###     CODE FOR YEAR, TITLE and MOVIEID    #######
movie_new=[]
for t,mid in zip(movies_data['title'],movies_data['movieId']):
    title = munge_title(t)
    year = get_year(t)
    movie_new.append([mid,title,year])

In [21]:
#movie_new

In [22]:
movieDf = pd.DataFrame(movie_new, columns=["movieId","title","year"])

In [23]:
movieDf.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [38]:
len(movieDf)-326

57772

In [36]:
movieDf[movieDf['year']==0]

,movieId,title,year
10023,32930,Category 6: Day of Destruction,0
10613,40697,Babylon 5,0
15719,79607,The Millions Game,0
17444,87442,"Bicycle, Spoon, Apple",0
24089,112406,Brazil: In the Shadow of the Stadiums,0
...,...,...,...
57504,192339,The Sign of Three,0
57674,192829,Parwaaz Hai Junoon,0
57789,193149,¿Qué te juegas?,0
57903,193443,Wall,0


In [40]:
movies_filtered = movieDf[movieDf['year'] != 0]
len(movies_filtered)
## keep movies_filtered ,, join later on

57772

In [35]:
len(movieDf)-326

57772

In [24]:
from imdb import Cinemagoer

In [29]:
nan_movies=pd.read_csv("Data/nan_movies.csv")
nan_movies.shape

(326, 3)

In [37]:
nan_movies

,Unnamed: 0,movieId,title
0,0,32930,Category 6: Day of Destruction
1,1,40697,Babylon 5
2,2,79607,The Millions Game
3,3,87442,"Bicycle, Spoon, Apple"
4,4,112406,Brazil: In the Shadow of the Stadiums
...,...,...,...
321,321,192339,The Sign of Three
322,322,192829,Parwaaz Hai Junoon
323,323,193149,¿Qué te juegas?
324,324,193443,Wall


In [56]:
len(nan_movies.iloc[0:210])

210

In [57]:
len(nan_movies.iloc[210:])

116

In [58]:
nan_movies_new=nan_movies.iloc[210:]
nan_movies_new

,Unnamed: 0,movieId,title
210,210,174039,Genesis
211,211,174447,A Gray State
212,212,174485,Salad Fingers
213,213,174707,Le Ride
214,214,174953,Mundo Cão
...,...,...,...
321,321,192339,The Sign of Three
322,322,192829,Parwaaz Hai Junoon
323,323,193149,¿Qué te juegas?
324,324,193443,Wall


In [59]:
## Get movie year from IMDB
imdb_year_1 = []
count=0
# create an instance of the Cinemagoer class
ia = Cinemagoer()

for x,m in zip(nan_movies_new['movieId'],nan_movies_new['title']):
    y = links[links['movieId']==x]['imdbId'].values[0]
     #################### DO NOT TOUCH THIS CODE #####################
    # get a movie
    movie = ia.get_movie(y)
    try:
        imdb_year_1.append([x,m,movie['year']])
    except:
        print("Id:",x,"Movie:",m,"ImdbId:",y)
        imdb_year_1.append([x,m,0])
    if (count%10==0):
        print(count,"------- Done ----------")
        print(m,movie['year'])
    count+=1
    
    ################## DO Not Touch this code --------########

0 ------- Done ----------
Genesis 2016
10 ------- Done ----------
All Superheroes Must Die 2: The Last Superhero 2016
20 ------- Done ----------
In Search of Fellini 2017
Id: 177265 Movie: Checkmate ImdbId: 5640954
30 ------- Done ----------
Sprawling from Grace 2008
40 ------- Done ----------
Going for Broke 2003
50 ------- Done ----------
Inside John Lennon - Unauthorized 2003
60 ------- Done ----------
Basmati Blues 2017
70 ------- Done ----------
Seat 25 2017
80 ------- Done ----------
Diamond Dogs 2017
Id: 188661 Movie: Untitled Star Trek Sequel ImdbId: 4756228
90 ------- Done ----------
Veras Mantel 2017
100 ------- Done ----------
Tell Me Your Name 2018
110 ------- Done ----------
Dansa först 2018


In [60]:
len(imdb_year)+len(imdb_year_1)

326

In [61]:
nan_movies_year = pd.DataFrame(imdb_year, columns=["movieId","title","year"])
nan_movies_year_1 = pd.DataFrame(imdb_year_1, columns=["movieId","title","year"])

In [62]:
movieFinal= pd.concat([movies_filtered, nan_movies_year,nan_movies_year_1], axis=0)

In [71]:
movieFinal.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [72]:
movieFinal.shape

(58098, 3)

In [73]:
print(len(movieFinal["movieId"].unique()))

58098


In [74]:
movieFinal[movieFinal["year"]==0]

,movieId,title,year
21,177265,Checkmate,0
88,188661,Untitled Star Trek Sequel,0


In [75]:
movieFinal.to_csv("movies2.csv",index=False)

In [76]:
df = pd.read_csv("movies2.csv")

In [77]:
df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [81]:
df.sort_values(by=['movieId'])

,movieId,title,year
57762,193866,Tales from the Hood 2,2018
57763,193868,Dos tipos de cuidado,1953
57764,193870,Isabelle Eberhardt,1991
57765,193872,Room Laundering,2018
57766,193874,Blondie's Big Moment,1947
57767,193876,The Great Glinka,1946
57768,193878,Les tribulations d'une caissière,2011
57769,193880,Her Name Was Mumu,2016
57770,193882,Flora,2017
57771,193886,Leal,2018


In [69]:
links[links['movieId']==177265]

,movieId,imdbId,tmdbId
50874,177265,5640954,468479.0


In [70]:
#pip install tmdb3


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for tmdb3: started
  Running setup.py install for tmdb3: finished with status 'done'


  DEPRECATION: tmdb3 is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [47]:
no_year_index = []
for i in range(len(year_list)):
    if year_list[i] == 0:
        no_year_index.append(i)  

In [49]:
len(no_year_index)

326

In [52]:
no_year_movie = []
no_year_movie_id = []
for year in no_year_index:
    no_year_movie.append(new_title[year])
    no_year_movie_id.append(movie_id_list[year])
    

In [51]:
no_year_movie

['Category 6: Day of Destruction',
 'Babylon 5',
 'The Millions Game',
 'Bicycle, Spoon, Apple',
 'Brazil: In the Shadow of the Stadiums',
 'Slaying the Badger',
 'Tatort: Im Schmerz geboren',
 'Terrible Joe Moran',
 'The Court-Martial of Jackie Robinson',
 'In Our Garden',
 'Stephen Fry In America - New World',
 'Two: The Story of Roman & Nyro',
 'A Year Along the Abandoned Road',
 'Body/Cialo',
 'Polskie gówno',
 'The Third Reich: The Rise & Fall',
 'La vendetta dei barbari',
 'My Own Man',
 'Moving Alan',
 'Zone 261',
 'Michael Laudrup - en Fodboldspiller',
 'Honky',
 'Blueberry Hill',
 'Pleasure Palace',
 'Terror on the 40th Floor',
 "My Boyfriend's Back",
 'One Night Only',
 'Doli Saja Ke Rakhna',
 'The Dead Lands',
 "C'mon, Let's Live a Little",
 'For a Book of Dollars',
 'Bad Boys 3',
 'Twins from Texas',
 'The Moon and the Sun',
 'Señorita Justice',
 'Red Victoria',
 'Vaastupurush',
 "Sierra Leone's Refugee All Stars",
 "L'uomo della carità",
 'The Big Fall',
 'Wolves In The Sn

In [53]:
len(no_year_movie_id)

326

In [58]:
nan_movies = pd.DataFrame({'movieId':no_year_movie_id,'title':no_year_movie})

In [59]:
#movie title that have no year
nan_movies

,movieId,title
0,32930,Category 6: Day of Destruction
1,40697,Babylon 5
2,79607,The Millions Game
3,87442,"Bicycle, Spoon, Apple"
4,112406,Brazil: In the Shadow of the Stadiums
...,...,...
321,192339,The Sign of Three
322,192829,Parwaaz Hai Junoon
323,193149,¿Qué te juegas?
324,193443,Wall


In [60]:
nan_movies.to_csv('nan_movies.csv')

In [61]:
#creating the table(title and year)
new_movie_title_year = pd.DataFrame({'movieId':movie_id_list,'title':new_title,'year':year_list})

In [62]:
new_movie_title_year

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995
...,...,...,...
58093,193876,The Great Glinka,1946
58094,193878,Les tribulations d'une caissière,2011
58095,193880,Her Name Was Mumu,2016
58096,193882,Flora,2017


In [70]:
df

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5
...,...,...,...
27753439,283228,8542,4.5
27753440,283228,8712,4.5
27753441,283228,34405,4.5
27753442,283228,44761,4.5


In [94]:
df.isna().sum()

userId           0
movieId          0
rating           0
y_unit_scaled    0
dtype: int64

In [71]:
userid_list = list(df['userId'])
rating_movieid_list = list(df['movieId'])
rating_list = list(df['rating'])

In [72]:
unique_movieid = set(rating_movieid_list)

In [76]:
print(len(rating_movieid_list))
print(len(unique_movieid))

27753444
53889


0


In [81]:
n_ratings = len(df)
n_train = math.floor(n_ratings)
itrain = df.index[:n_train]
ival = df.index[n_train:]

In [84]:
df['y'] = df['rating'] - df.loc[itrain, 'rating'].mean()

In [88]:
SCALE = 1
if SCALE:
    yscaler = sklearn.preprocessing.MinMaxScaler()
    yscaler.fit(df.loc[itrain, 'rating'].values.reshape(-1, 1))
    df['y_unit_scaled'] = yscaler.transform(df['rating'].values.reshape(-1, 1))

In [91]:
df.drop('y',axis = 1,inplace = True)